<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.2
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerous than some, and does not have waiting period for approval of new developers.

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)


- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access
- fill in the form fields at the bottom
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"

- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [112]:
!pip install praw

In [113]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [116]:
my_user_agent = 'anna_app:v1.0 (by /u/Worried-Character167)'   # your user Agent string goes in here
my_client_id = 'FO7NKfHilhnSarFC2rQyrg'   # your Client ID string goes in here
my_client_secret = 'Zp4Z8SuDoIOle82-ONA9_t46tgb4vQ'   # your Secret string goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [121]:
pwd()  # make sure your working directory is where the file is

'/Users/annaxu/Documents/Data Science/Week 3/Labs'

In [123]:
path_auth = '/Users/annaxu/Documents/Data Science/Week 3/Labs/auth_reddit.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:

pp.pprint(auth)

{   'my_client_id': 'FO7NKfHilhnSarFC2rQyrg',
    'my_client_secret': 'Zp4Z8SuDoIOle82-ONA9_t46tgb4vQ',
    'my_user_agent': 'anna_app:v1.0 (by /u/Worried-Character167)'}


In [125]:
my_user_agent = auth['my_user_agent']
my_client_id = auth['my_client_id']
my_client_secret = auth['my_client_secret']

Security considerations:
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [130]:
reddit = praw.Reddit(client_id = my_client_id,
                     client_secret = my_client_secret,
                     user_agent = my_user_agent)

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [133]:
reddit.

SyntaxError: invalid syntax (1019240351.py, line 1)

In [154]:
subreddit_name = 'foodies_sydney'
subreddit = reddit.subreddit(subreddit_name)

In [156]:
comments = []
for comment in subreddit.comments(limit=1000):
    comments.append(comment)

In [157]:
reddit.subreddit(subreddit_name)

Subreddit(display_name='foodies_sydney')

In [163]:
# Get information about the subreddit
subreddit = reddit.subreddit(subreddit_name)

# Print some basic information about the subreddit
print(f"Subreddit Name: {subreddit.display_name}")
print(f"Subreddit Title: {subreddit.title}")
print(f"Subreddit Description: {subreddit.description}")
print(f"Number of Subscribers: {subreddit.subscribers}")

Subreddit Name: foodies_sydney
Subreddit Title: foodies_sydney
Subreddit Description: 
Number of Subscribers: 33310


Consult the PRAW and Reddit API documentation. Print a few of the response members below:

In [167]:
# Print the top posts in the subreddit
top_posts = subreddit.top(limit=5)  # You can change the limit to fetch more posts
print("\nTop Posts:")
for post in top_posts:
    print(f"Title: {post.title}")
    print(f"Author: {post.author}")
    print(f"Score: {post.score}")
    print(f"URL: {post.url}\n")


Top Posts:
Title: A cafe closure to celebrate!!!
Author: beepbop213
Score: 2250
URL: https://i.redd.it/lfsl8ylbcyye1.jpeg

Title: All the wonderful food we had on our Sydney trip
Author: earth_wanderer1235
Score: 725
URL: https://www.reddit.com/gallery/1jcdh1q

Title: Pretty but would you classify as scrambled eggs?
Author: lifeis_amystery
Score: 699
URL: https://www.reddit.com/gallery/1j2wbmh

Title: After supermarkets they should investigate Sydney restaurants 🤷🏽‍♂️
Author: privatepolicy85
Score: 575
URL: https://i.redd.it/jm8jswp4w5fc1.jpeg

Title: This is a fun little excercise😂
Author: None
Score: 518
URL: https://i.redd.it/74457q7bgnkc1.jpeg



Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic.

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [169]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Ask Anything Monday - Weekly Thread
How can I keep up to date with the hottest python modules?
Can’t Pass Arg into Run Script via SSH shortcut Action
How to use variables in other libraries
How to bind semi_annual_raise correctly with while loop and if condition
Anyone know how I can make a 3D game like this? (Read desc)
How can python help me in a career?
New to VSC and the terminal and utterly confused
unsupported file
Plotting advice


Now retrieve 10 authors:

In [172]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author)

AutoModerator
MrMrsPotts
DrMistyDNP
BISACS
DigitalSplendid
CoolCademM
ItsDanny201
Level_String6853
tommzo28
FeistyAssumption3237


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

In [175]:
submissions=reddit.subreddit('learnpython')

Why doesn't the next cell produce output?

In [178]:
for submission in submissions:
    print(submission.comments)

TypeError: 'Subreddit' object is not iterable

In [180]:
#Submission returns a Subreddit object, not a list of submissions that can be iterated on

Print two comments associated with each of these submissions:

In [183]:
submissions = reddit.subreddit('learnpython').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

I am very new to coding. I tried to write a simple "whack-a-mole" type game that displays a 3x3 matrix of X's and you press the key on the number pad which corresponds to the "D" which replaces one of the X's. For troubleshooting purposes, it also prints the number you need to press. 

When I run it, the code does what I expect and stops to wait for a button press. Then it randomly selects another X matrix from the list, generally marks it as a wrong answer using the initial numberpad input, then prints a third matrix from the list and stops to wait for another input. I do not understand where the second selection from the list is coming from? 

I added an unused item to my list of matrices so I could just call positions 1-9 to correspond to the numbers on the number pad. I understand I probably could have written it as number\_key\_to\_press = list\_value + 1 but I do not think that this has anything to do with the problem I am actually facing with my code. 

    import random
    imp

Referring to the API documentation, explore the submissions object and print some interesting data:

#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request:

In [192]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

False


You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

>
>


>
>




---



---



> > > > > > > > > © 2025 Institute of Data


---



---



